<a href="https://colab.research.google.com/github/LakshmiMAchar03/NEW_REPO/blob/master/ML_Model_classification_Modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install pyspark

In [17]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("BankMarketingClassification").getOrCreate()

In [18]:
# Load the dataset
df = spark.read.csv("/content/bank.csv", header=True, inferSchema=True)

In [19]:
df.show(10)

+---+----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|    admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|    admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41|technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|  services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|    admin.| married| tertiary|     no|    184|     no|  no|unkno

In [20]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [21]:
df = df.na.drop()  # Drop rows with nulls


In [22]:
df.show(3)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
only showing top 3 rows



In [23]:
# Convert categorical columns (job, marital, education, etc.) into numerical values using StringIndexer
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=col, outputCol=col+"_index") for col in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']]



In [24]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

pipeline.fit(df) fits the indexers to the data (assigning numbers to categories).

.transform(df) transforms the DataFrame by adding new columns with indexed values.



In [25]:
df.show(3)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|job_index|marital_index|education_index|default_index|housing_index|loan_index|contact_index|month_index|poutcome_index|deposit_index|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|      3.0|          0.0|            0.0|          0.0|          1.0|   

In [26]:
#Combine all features into a single vector using VectorAssembler.
from pyspark.ml.feature import VectorAssembler

feature_columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'job_index', 'marital_index',
                   'education_index', 'default_index', 'housing_index', 'loan_index', 'contact_index', 'month_index', 'poutcome_index']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)


In [27]:
df.show(3)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+--------------------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|job_index|marital_index|education_index|default_index|housing_index|loan_index|contact_index|month_index|poutcome_index|deposit_index|            features|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+--------------------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|      3.

In [28]:
#Choose the features column and the deposit_index column as the label.
df = df.select("features", "deposit_index")

In [29]:
train_df, test_df = df.randomSplit([0.8, 0.2])


Logistic Regression

In [30]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'deposit_index', maxIter=10)
lrModel = lr.fit(train_df)

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Make predictions
predictions = lrModel.transform(test_df)

In [35]:

# Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="deposit_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7987278509768287


RF

In [31]:
from pyspark.ml.classification import RandomForestClassifier
# Create Random Forest model
rf = RandomForestClassifier(labelCol="deposit_index", featuresCol="features", numTrees=100)
# Train the model
rf_model = rf.fit(train_df)


In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Make predictions
predictions = rf_model.transform(test_df)

In [33]:
# Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="deposit_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8123580190822354
